In [52]:
import pandas as pd 
import numpy as np

#Matrix operations
from itertools import product

# Variable Elimination for the moralized graph

Implement inference algorithms for computing the conditional distribution of $X_{10}$ and $X_{11}$ given the observed variables for any choice of parameters $\psi^{(1)}, \ldots, \psi^{(11)}$.

### Synthetic data

In [53]:
alpha = 0.5

Elemination order

In [54]:
# The scope consist of the parents and the node itself
variables_to_eliminate = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9']
elimination_scope = [("X1", ['X7', 'X8']),("X2", ['X4', 'X8']),("X3", ['X9', 'X8']),
                     ("X5", ['X9', 'X8']),("X4", ['X11', 'X8', 'X9']),("X6", ['X7', 'X10']),
                     ("X7", []),("X8", []),("X9", ['X10', 'X6']), ("X10", ['X7']),("X11", ['X8', 'X9'])]


### From CPD to factor representation

In [55]:
def create_factor(variable,parent_list):
    d = 1 + len(parent_list)
    columns = parent_list + [variable]
    df = pd.DataFrame(product([0,1],repeat=d),columns=columns)
    return df

def cpd_factor(variable, parent_list, alpha):
    cpd = create_factor(variable, parent_list)
    variables = list(cpd.columns)
    n = len(variables)
    cpd_ = cpd.copy() 
    if n == 1:
        cpd_["prob"] = 1 - alpha
        return cpd_
    else:
        child = variables[n-1]
        # generate factor 
        parents = cpd_.iloc[:,:-1].copy()
        parents["sum"] = parents.sum(axis=1)
        cpd_["prob"] = 1 - (alpha ** (1 + parents["sum"]))
        # Update the probability where condition false
        # St. we can compute the probability of P(X=0|parents) = 1 - P(X=1|parents)
        cpd_["prob"].where(cpd_[f"{child}"] == 1, 1-cpd_["prob"],axis=0,inplace=True)
        return cpd_
    
factor_test = cpd_factor(elimination_scope[2][0], elimination_scope[2][1], alpha)
factor_test


,X9,X8,X3,prob
0,0,0,0,0.500
1,0,0,1,0.500
2,0,1,0,0.250
3,0,1,1,0.750
4,1,0,0,0.250
5,1,0,1,0.750
6,1,1,0,0.125
7,1,1,1,0.875


In [56]:
# Generate factors for all nodes
# list form: [(factor, protein name), ... ]
factor_list = []
for elm in elimination_scope:
    name = 'phi_' + str(elm[0])
    cpd = cpd_factor(elm[0], elm[1], alpha)
    factor_list.append((cpd, name))

print(factor_list[0])

(   X7  X8  X1   prob
0   0   0   0  0.500
1   0   0   1  0.500
2   0   1   0  0.250
3   0   1   1  0.750
4   1   0   0  0.250
5   1   0   1  0.750
6   1   1   0  0.125
7   1   1   1  0.875, 'phi_X1')


# Sum product

In [57]:
# Multiply factors togheter
def factor_product(factor_1, factor_2):
    # List of all common nodes to join on
    nodes_to_join = factor_1.columns.intersection(factor_2.columns)
    nodes_to_join = nodes_to_join.drop("prob")
    nodes = list(nodes_to_join)

    #Inner join the factors by common nodes
    new_factor = factor_1.merge(factor_2, how='inner', on=nodes)
    new_factor["prob"] = new_factor["prob_x"]*new_factor["prob_y"] # Mulætiply the probabilities
    new_factor = new_factor.drop(columns=["prob_x","prob_y"])
    return new_factor

factor_product(factor_list[0][0], factor_list[1][0])

,X7,X8,X1,X4,X2,prob
0,0,0,0,0,0,0.250000
1,0,0,0,0,1,0.250000
2,0,0,0,1,0,0.125000
3,0,0,0,1,1,0.375000
4,0,0,1,0,0,0.250000
5,0,0,1,0,1,0.250000
6,0,0,1,1,0,0.125000
7,0,0,1,1,1,0.375000
8,1,0,0,0,0,0.125000
9,1,0,0,0,1,0.125000


In [58]:
# From algorithm 9.1 in the book (p. 298)
def sum_product_eliminate_var(factors : list, var : int, print_info = False):
    factors_list = [i[0] for i in factors] # Factors
    names_list = [i[1] for i in factors] # Names of factors

    lst_scope = []   # Factors in scope
    lst_scope_i = [] # Names of factors in scope   
    lst_not_scope = [] # Factors not in scope
    lst_not_scope_i = [] # Names of factors not in scope

    # Split factors into scope and not scope
    for i in range(0, len(factors_list)):
        if var in factors_list[i].columns:
            lst_scope.append(factors_list[i])
            lst_scope_i.append(names_list[i])
        else:
            lst_not_scope.append(factors_list[i])
            lst_not_scope_i.append(names_list[i])

    # Multiply all factors in scope
    factor_ = lst_scope[0]
    for i in range(1, len(lst_scope)):
        factor_ = factor_product(factor_, lst_scope[i])

    # All variables that should not be marginalized
    col = list(factor_.columns[factor_.columns != var].drop("prob")) 

    # Marginalize the variable
    tau = factor_.groupby(col, as_index = False)['prob'].sum()
    tau_name = 'tau_' + str(var) 

    # Add the new factor to the list of factors not in scope
    lst_not_scope = lst_not_scope + [tau]
    lst_not_scope_i = lst_not_scope_i + [tau_name]
    
    # Debuggering prints
    if print_info:
        for elm in lst_scope:
            print(elm.columns)
        print(lst_scope_i)

    return list(zip(lst_not_scope, lst_not_scope_i))

print(sum_product_eliminate_var(factor_list, 'X1', False))

[(   X4  X8  X2   prob
0   0   0   0  0.500
1   0   0   1  0.500
2   0   1   0  0.250
3   0   1   1  0.750
4   1   0   0  0.250
5   1   0   1  0.750
6   1   1   0  0.125
7   1   1   1  0.875, 'phi_X2'), (   X9  X8  X3   prob
0   0   0   0  0.500
1   0   0   1  0.500
2   0   1   0  0.250
3   0   1   1  0.750
4   1   0   0  0.250
5   1   0   1  0.750
6   1   1   0  0.125
7   1   1   1  0.875, 'phi_X3'), (   X9  X8  X5   prob
0   0   0   0  0.500
1   0   0   1  0.500
2   0   1   0  0.250
3   0   1   1  0.750
4   1   0   0  0.250
5   1   0   1  0.750
6   1   1   0  0.125
7   1   1   1  0.875, 'phi_X5'), (    X11  X8  X9  X4    prob
0     0   0   0   0  0.5000
1     0   0   0   1  0.5000
2     0   0   1   0  0.2500
3     0   0   1   1  0.7500
4     0   1   0   0  0.2500
5     0   1   0   1  0.7500
6     0   1   1   0  0.1250
7     0   1   1   1  0.8750
8     1   0   0   0  0.2500
9     1   0   0   1  0.7500
10    1   0   1   0  0.1250
11    1   0   1   1  0.8750
12    1   1   0   0  0.1250


In [59]:
def sum_product_ve(factors, order):

    # For each variable in the elimination order
    for var in order:
        factors = sum_product_eliminate_var(factors, var)
    return factors

print(sum_product_ve(factor_list, variables_to_eliminate))

[(   X11  X10      prob
0    0    0  0.095215
1    0    1  0.134399
2    1    0  0.279785
3    1    1  0.490601, 'tau_X9')]


In [60]:
variables_to_eliminate = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9']
K = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11']
evidence = [('X1', 1), ('X2',1), ('X3',1),('X4',1),("X5",1),('X6',1),('X7',1),('X8',1),('X9',0)]
query_var = ['X10', 'X11']

# k - parameterizing nework k over X
# query_var - Set of query variables Y
# order - Elimination order (order)
# evidence - observed values e for variables in E
def cond_prob_ve (factors: list, queries: list, evidence: list, order: list):
    order_ = order.copy() # Fix for wierd python handeling of list
    factors_list = [i[0] for i in factors] # Factors
    names_list = [i[1] for i in factors] # Names of factors

    for i in range(0,len(factors_list)): # Loop through factors
        for elm in evidence: # Loop through evidence
            if elm[0] in factors_list[i].columns:
                # Retrict factor to evidence == e, and replace in factor list
                factors_list[i] = factors_list[i][factors_list[i][elm[0]]==elm[1]]
    
    # Remove elemetns we want to keep from elemination order
    # for elm in evidence:
    #     order_.remove(elm)
    
    concat_factor_list = list(zip(factors_list,names_list))
    #print(len(factors_list), len(names_list), '\n', names_list)
    
    phi = sum_product_ve(concat_factor_list, order_)
    #print(phi)

    alpha = phi[0][0].groupby(queries, as_index = False)['prob'].sum()
    alpha['prob'] = alpha['prob'] / alpha['prob'].sum() # Normalize the probability
   
    return alpha, phi

cond_prob_ve(factor_list, query_var, evidence, variables_to_eliminate)

(   X10  X11      prob
 0    0    0  0.080808
 1    0    1  0.282828
 2    1    0  0.141414
 3    1    1  0.494949,
 [(   X11  X10      prob
   0    0    0  0.000946
   1    0    1  0.001656
   2    1    0  0.003312
   3    1    1  0.005796,
   'tau_X9')])